In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, linear_model, metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

C:\Users\yuranich\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("orange_small_churn_train_data.csv", index_col='ID')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Columns: 231 entries, Var1 to labels
dtypes: float64(191), int64(2), object(38)
memory usage: 70.8+ MB


Проведем предобработку, аналогичную той, что делали для бейзлайна.

In [3]:
numbers = train.ix[:, 0:190]
categors = train.ix[:, 190:230]
numbers.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.0,NaN
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
categors.head()

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NESt0G8EIb,AERks4l,NaN,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,NaN,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,FoxgUHSK8h,LrdZy8QqgUfkVShG,NaN,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,vNEvyxLp3Z,RO12,NaN,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,NaN,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,4e7gUH7IEC,RO12,NaN,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,NaN,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


In [5]:
names_to_drop = []
for name in numbers.columns:
    col = numbers[name].dropna()
    if len(col.unique()) < 2:
        names_to_drop.append(name)

numbers.drop(names_to_drop, axis=1, inplace=True)
numbers.fillna(0, inplace=True)
numbers.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
ID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,3052.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1813.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,276.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1953.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1533.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,686.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
names_to_drop = []
for name in categors.columns:
    col = categors[name].dropna()
    if len(col.unique()) < 2:
        names_to_drop.append(name)

categors.drop(names_to_drop, axis=1, inplace=True)
categors.fillna(0, inplace=True)
categors.head()

,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,...,Var219,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229
ID,,,,,,,,,,,,,,,,,,,,,
0,NESt0G8EIb,AERks4l,0,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,0,0,...,FzaX,hLKtJ9p,Al6ZaUT,vr93T2a,LM8l689qOp,0,fKCe,02N6s8f,xwM2aC7IdeMC0,0
1,P1WvyxLp3Z,2Knk1KF,0,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,0,...,FzaX,hN8KpA1,oslk,6hQ9lNX,LM8l689qOp,ELof,xb3V,RAYp,55YFVY9,mj86
2,FoxgUHSK8h,LrdZy8QqgUfkVShG,0,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,0,0,...,FzaX,4UxGlow,zCkv,catzS2D,LM8l689qOp,0,FSa2,ZI9m,ib5G6X1eUxUn6,mj86
3,vNEvyxLp3Z,RO12,0,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,0,0,...,FzaX,ylCK5YS,oslk,e4lqvY0,LM8l689qOp,0,xb3V,RAYp,F2FyR07IdsN7I,0
4,4e7gUH7IEC,RO12,0,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,0,0,...,FzaX,PYpzAu9,oslk,MAz3HNj,LM8l689qOp,0,WqMG,RAYp,F2FyR07IdsN7I,0


Бейзлайн показал довольно плохое качество, особенно по полноте. Преодолеть его скорее всего будет довольно легко, поэтому сначала поэкспериментируем. Оценим качество модели по кроссвалидации используя только числовые признаки. В качестве классификатора пока что возьмем Random Forest. Метрику качества сразу будем использовать AUC-PRC

In [20]:
y = train['labels']
cv_strategy = cross_validation.StratifiedKFold(y, random_state = 2)
forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), numbers, y, scoring = 'average_precision', cv = cv_strategy)
print forest_scoring.mean(), ", ", forest_scoring.min(), ", ", forest_scoring.max()

0.10990613099 ,  0.105582808828 ,  0.114598944622


Результат очень слабый, поработаем с числовыми признаками плотнее. Отмасштабируем и возьмем только наиболее коррелирующие.

In [24]:
numbers = (numbers - numbers.min()) / (numbers.max() - numbers.min())
numbers.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.000293,0.000025,0.000083,0.000175,0.001199,0.008907,0.17252,0.000294,0.000901,0.006712,...,0.003815,0.011211,0.003724,0.000666,0.000187,0.000491,0.000251,0.016649,0.176861,0.000653
std,0.008149,0.005000,0.005445,0.009278,0.020289,0.018941,0.18001,0.008013,0.013952,0.043646,...,0.044647,0.048386,0.039513,0.011882,0.005613,0.011740,0.007501,0.049584,0.225655,0.013508
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.002178,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010110,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.006056,0.20000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010110,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.009988,0.20000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010110,0.383178,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
#Проверим, что само по себе масштабирование особо не влияет.
forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), numbers, y, scoring = 'average_precision', cv = cv_strategy)
forest_scoring.mean()

0.10680133068390441

In [25]:
corrs = []
size1 = float(y[y == 1].size)
size_1 = float(y[y == -1].size)

for name in numbers.columns:
    col = numbers[name]
    corr = abs(col[y == 1].mean() / size1 - col[y == -1].mean() / size_1)
    corrs.append((corr, name))
    
len(corrs)

173

In [26]:
sorted(corrs, reverse=True)[:10]

[(0.00015366476529006388, 'Var113'),
 (0.00015357213284949964, 'Var57'),
 (0.00013249708678350096, 'Var153'),
 (0.00010748832088920868, 'Var126'),
 (5.746082180149363e-05, 'Var73'),
 (4.723032507904624e-05, 'Var189'),
 (4.4503258578500575e-05, 'Var133'),
 (4.146724723635516e-05, 'Var7'),
 (4.091718707992928e-05, 'Var38'),
 (3.320716483984746e-05, 'Var144')]

In [59]:
corr_nums = numbers[[x[1] for x in sorted(corrs, reverse=True)[:5]]]
forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), corr_nums, y, scoring = 'average_precision', cv = cv_strategy)
forest_scoring.mean()

0.13262073641669281

Видим улучшение качества, хотя и незначительное. Теперь поработаем отдельно с категориальными признаками.

In [60]:
import scipy.stats as ss

def cramers_stat(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))

def get_conf_matrix(feature, target):
    confusions = []
    categories = feature.unique()
    for i in categories:
        for j in [-1, 1]:
            one = (feature == i).values
            two = target == j
            cond = one & two
            confusions.append(cond.sum())
            
    return np.array(confusions).reshape(len(categories), 2)

In [61]:
cat_corr = []
for name in categors.columns:
    col = categors[name]
    conf_mat = get_conf_matrix(col, y)
    cr_stat = cramers_stat(conf_mat)
    cat_corr.append(cr_stat)
    
len(cat_corr)

34

In [62]:
corrs_cat = zip(cat_corr, categors.columns)
sorted(corrs_cat, reverse=True)[:10]

[(0.52986260512218375, 'Var217'),
 (0.52599126441625521, 'Var214'),
 (0.52599126441625521, 'Var200'),
 (0.40783117717402112, 'Var199'),
 (0.36343287292663434, 'Var202'),
 (0.29259328641845067, 'Var222'),
 (0.29259328641845067, 'Var220'),
 (0.29259328641845067, 'Var198'),
 (0.22894003457385523, 'Var216'),
 (0.15124744994612724, 'Var192')]

Для тех признаков, где мало категорий, просто перекодируем их в бинарные

In [64]:
cat_dums = pd.get_dummies(categors[[x for x in categors.columns if len(categors[x].unique()) < 10]])
cat_dums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Data columns (total 63 columns):
Var194_0             40000 non-null float64
Var194_CTUH          40000 non-null float64
Var194_SEuy          40000 non-null float64
Var194_lvza          40000 non-null float64
Var196_1K8T          40000 non-null float64
Var196_JA1C          40000 non-null float64
Var196_mKeq          40000 non-null float64
Var196_z3mO          40000 non-null float64
Var201_0             40000 non-null float64
Var201_6dX3          40000 non-null float64
Var201_smXZ          40000 non-null float64
Var203_0             40000 non-null float64
Var203_9_Y1          40000 non-null float64
Var203_F3hy          40000 non-null float64
Var203_HLqf          40000 non-null float64
Var203_dgxZ          40000 non-null float64
Var203_pybr          40000 non-null float64
Var205_0             40000 non-null float64
Var205_09_Q          40000 non-null float64
Var205_VpdQ          40000 non-null float64
Var205_sJzT

Остальные преобразуем в числовые по количеству соответствий классу 1.

In [66]:
cat_copy = categors.copy()
for name in cat_copy.columns:
    if len(cat_copy[name].unique()) >= 10:
        counts = {}
        pairs = zip(cat_copy[name].values, y)
        for x in pairs:
            if counts.get(x[0]) == None:
                    counts[x[0]] = 0
            if x[1] == 1:
                counts[x[0]] += 1
        cat_copy[name] = cat_copy[name].apply(lambda x: counts[x])
        print cat_copy[name][:10]


ID
0    15
1    18
2    29
3     9
4    22
5    20
6    23
7    22
8     8
9    23
Name: Var192, dtype: int64
ID
0     102
1     268
2      10
3    2433
4    2433
5    2433
6    2433
7    2433
8    2433
9    2433
Name: Var193, dtype: int64
ID
0    2896
1    2896
2    2896
3    2896
4    2896
5    2896
6    2896
7    2896
8    2896
9    2896
Name: Var195, dtype: int64
ID
0      7
1      9
2    273
3    311
4     15
5      8
6    311
7     17
8    273
9     21
Name: Var197, dtype: int64
ID
0      0
1      0
2    138
3      4
4      0
5      0
6      4
7      4
8      8
9      1
Name: Var198, dtype: int64
ID
0     2
1    10
2     5
3     8
4     2
5     3
6     0
7     4
8    24
9    14
Name: Var199, dtype: int64
ID
0    1794
1       0
2    1794
3    1794
4    1794
5    1794
6    1794
7    1794
8    1794
9    1794
Name: Var200, dtype: int64
ID
0    0
1    0
2    3
3    1
4    0
5    0
6    2
7    4
8    3
9    0
Name: Var202, dtype: int64
ID
0    106
1     39
2     63
3     65
4     81
5 

In [67]:
cat_copy = cat_copy.drop([x for x in categors.columns if len(categors[x].unique()) < 10], axis=1)
cat_copy.head()

,Var192,Var193,Var195,Var197,Var198,Var199,Var200,Var202,Var204,Var206,Var207,Var212,Var214,Var216,Var217,Var219,Var220,Var222,Var226,Var228
ID,,,,,,,,,,,,,,,,,,,,
0,15,102,2896,7,0,2,1794,0,106,1269,65,38,1794,11,0,2443,0,0,154,88
1,18,268,2896,9,0,10,0,0,39,97,2334,251,0,4,0,2443,0,0,132,171
2,29,10,2896,273,138,5,1794,3,63,106,245,27,1794,0,2,2443,138,138,607,85
3,9,2433,2896,311,4,8,1794,1,65,1269,2334,2127,1794,243,1,2443,4,4,132,2270
4,22,2433,2896,15,0,2,1794,0,81,518,2334,2127,1794,39,0,2443,0,0,248,2270


Посмотрим на результаты только на основе этих признаков.

In [73]:
forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), cat_copy, y, scoring = 'average_precision', cv = cv_strategy)
forest_scoring.mean()

0.59681281993831392

Результат уже значительно выше, чем для числовых. Добавим сюда бинаризированные признаки

In [69]:
all_cats = pd.concat([cat_dums, cat_copy], axis=1)
all_cats.head()

,Var194_0,Var194_CTUH,Var194_SEuy,Var194_lvza,Var196_1K8T,Var196_JA1C,Var196_mKeq,Var196_z3mO,Var201_0,Var201_6dX3,...,Var207,Var212,Var214,Var216,Var217,Var219,Var220,Var222,Var226,Var228
ID,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,65,38,1794,11,0,2443,0,0,154,88
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2334,251,0,4,0,2443,0,0,132,171
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,245,27,1794,0,2,2443,138,138,607,85
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2334,2127,1794,243,1,2443,4,4,132,2270
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2334,2127,1794,39,0,2443,0,0,248,2270


In [75]:
forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), all_cats, y, scoring = 'average_precision', cv = cv_strategy)
forest_scoring.mean()

0.61852467449436654

Результат почти не изменился. Попробуем отобрать теперь из них только признаки с наибольшей корреляцией.

In [76]:
def contains(names, name):
    for x in names:
        if x in name:
            return True
    return False

In [101]:
for i in [5,10,15,25,30,35]:
    names = [x[1] for x in sorted(corrs_cat, reverse=True)[:i]]
    curr_names = [x for x in all_cats.columns if contains(names, x)]
    forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), all_cats[curr_names], y, scoring = 'average_precision', cv = cv_strategy)
    print "score for %s best features: " % i, forest_scoring.mean()

score for 5 best features:  0.500114977637
score for 10 best features:  0.566339031803
score for 15 best features:  0.608641354659
score for 25 best features:  0.609504197062
score for 30 best features:  0.622317999559
score for 35 best features:  0.61648536409


Видим, что наилучшее качество достигается на 30, видимо дальше начинаются шумовые признаки. Объединим с лучшими числовыми.

In [102]:
names = [x[1] for x in sorted(corrs_cat, reverse=True)[:30]]
curr_names = [x for x in all_cats.columns if contains(names, x)]
filtered = pd.concat([numbers[[x[1] for x in sorted(corrs, reverse=True)[:5]]], all_cats[curr_names]], axis=1)
filtered.head()

,Var113,Var57,Var153,Var126,Var73,Var194_0,Var194_CTUH,Var194_SEuy,Var194_lvza,Var201_0,...,Var207,Var212,Var214,Var216,Var217,Var219,Var220,Var222,Var226,Var228
ID,,,,,,,,,,,,,,,,,,,,,
0,0.435428,0.415400,0.166373,0.36,0.115385,1.0,0.0,0.0,0.0,1.0,...,65,38,1794,11,0,2443,0,0,154,88
1,0.517911,0.838613,0.467556,0.72,0.476923,1.0,0.0,0.0,0.0,1.0,...,2334,251,0,4,0,2443,0,0,132,171
2,0.490419,0.854514,0.759936,0.68,0.623077,1.0,0.0,0.0,0.0,1.0,...,245,27,1794,0,2,2443,138,138,607,85
3,0.515912,0.729720,0.695739,0.32,0.100000,1.0,0.0,0.0,0.0,1.0,...,2334,2127,1794,243,1,2443,4,4,132,2270
4,0.503960,0.092932,0.757503,0.04,0.107692,1.0,0.0,0.0,0.0,1.0,...,2334,2127,1794,39,0,2443,0,0,248,2270


In [103]:
forest_scoring = cross_validation.cross_val_score(RandomForestClassifier(), filtered, y, scoring = 'average_precision', cv = cv_strategy)
forest_scoring.mean()

0.65045484417349442

Будем считать, что достигли на данном этапе приемливого по сравнению с бейзлайном качества, можно проверить на тесте и отправить первый результат в inclass

In [104]:
test = pd.read_csv("orange_small_churn_test_data.csv", index_col='ID')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 17.6+ MB


In [106]:
test.fillna(0, inplace=True)
test_nums = test[[x[1] for x in sorted(corrs, reverse=True)[:5]]]
test_nums.head()

,Var113,Var57,Var153,Var126,Var73
ID,,,,,
0,184566.8,3.887844,10278280.0,0.0,124
1,82004.4,5.320230,777520.0,0.0,44
2,203446.8,0.628285,5667840.0,-24.0,88
3,102326.8,3.509720,10506480.0,34.0,82
4,-4165080.0,6.719932,0.0,-20.0,4


In [109]:
test_cats = test.ix[:, 190:230]
test_dums = pd.get_dummies(test_cats[[x for x in test_cats.columns if len(test_cats[x].unique()) < 10]])
test_dums.head()

,Var209,Var230,Var191_0,Var191_r__I,Var194_0,Var194_CTUH,Var194_SEuy,Var194_lvza,Var196_1K8T,Var196_JA1C,...,Var227_RAYp,Var227_ZI9m,Var227_nIGXDli,Var227_nIGjgSB,Var227_vJ_w8kB,Var229_0,Var229_am7c,Var229_mj86,Var229_oJmt,Var229_sk2h
ID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [113]:
cat_copy = test_cats.copy()
cat_copy = cat_copy.drop([x for x in test_cats.columns if len(test_cats[x].unique()) < 10], axis=1)
for name in cat_copy.columns:
    counts = {}
    pairs = zip(cat_copy[name].values, y)
    for x in pairs:
        if counts.get(x[0]) == None:
                counts[x[0]] = 0
        if x[1] == 1:
            counts[x[0]] += 1
    cat_copy[name] = cat_copy[name].apply(lambda x: counts[x])
    print cat_copy[name][:10]

ID
0    4
1    1
2    4
3    3
4    0
5    3
6    2
7    3
8    5
9    0
Name: Var192, dtype: int64
ID
0    112
1    531
2    531
3    531
4    531
5    531
6    531
7     13
8    531
9    531
Name: Var193, dtype: int64
ID
0    739
1    739
2    739
3    739
4    739
5    739
6    739
7    739
8    739
9    739
Name: Var195, dtype: int64
ID
0    74
1    74
2     6
3    62
4    63
5     2
6     3
7     1
8     2
9    74
Name: Var197, dtype: int64
ID
0    18
1     3
2     1
3     1
4     1
5     0
6     1
7    74
8     0
9     0
Name: Var198, dtype: int64
ID
0     0
1     2
2    14
3     5
4     0
5     1
6     6
7     1
8     0
9     0
Name: Var199, dtype: int64
ID
0      0
1    388
2      0
3      1
4    388
5    388
6    388
7      0
8      0
9    388
Name: Var200, dtype: int64
ID
0    0
1    0
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: Var202, dtype: int64
ID
0    16
1    16
2     8
3     8
4     8
5     6
6    13
7    12
8     8
9    11
Name: Var204, dtype: int64


In [114]:
#не можем профильтровать бинаризированные категории, поскольку имена признаков отличаются. Пока без них.
test_data = pd.concat([test_nums, cat_copy], axis=1)
test_data.head()

,Var113,Var57,Var153,Var126,Var73,Var192,Var193,Var195,Var197,Var198,...,Var207,Var212,Var214,Var216,Var217,Var219,Var220,Var222,Var226,Var228
ID,,,,,,,,,,,,,,,,,,,,,
0,184566.8,3.887844,10278280.0,0.0,124,4,112,739,74,18,...,23,20,0,5,0,630,18,18,35,15
1,82004.4,5.320230,777520.0,0.0,44,1,531,739,74,3,...,527,444,388,8,0,630,3,3,35,486
2,203446.8,0.628285,5667840.0,-24.0,88,4,531,739,6,1,...,527,444,0,20,1,630,1,1,25,486
3,102326.8,3.509720,10506480.0,34.0,82,3,531,739,62,1,...,527,444,1,74,1,630,1,1,25,486
4,-4165080.0,6.719932,0.0,-20.0,4,0,531,739,63,1,...,527,444,388,4,1,630,1,1,34,486


In [117]:
clf = RandomForestClassifier()
clf.fit(filtered[test_data.columns], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [118]:
result = clf.predict_proba(test_data)
result[:10]

array([[ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.6,  0.4],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ]])

In [120]:
def write_result(result):
    churn_probs = [x[1] for x in result]
    ids = range(len(churn_probs))
    df = pd.DataFrame(index=ids, data=churn_probs, columns=['result'])
    df.to_csv("result.csv")

In [121]:
write_result(result)

Остановимся пока на 25 признаках и поиграимся с параметрами классификатора.

In [130]:
filtered = filtered[test_data.columns]
clf = RandomForestClassifier(random_state=2)
clf.get_params().keys()

['warm_start',
 'oob_score',
 'n_jobs',
 'verbose',
 'max_leaf_nodes',
 'bootstrap',
 'min_samples_leaf',
 'n_estimators',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'criterion',
 'random_state',
 'max_features',
 'max_depth',
 'class_weight']

In [127]:
parameters_grid = {
    'max_depth' : [5, 10, 15, 20, 25],
    'min_samples_leaf' : [1, 2, 3, 5, 8],
    'n_estimators' : [10, 50, 100, 300],
    'criterion' : ['gini', 'entropy'],
    'max_features' : [0.2, 0.4, 0.6, 0.7, 0.8, 0.9]
}

In [128]:
from sklearn import grid_search

In [129]:
grid_cv = grid_search.GridSearchCV(clf, parameters_grid, scoring = 'average_precision', cv = cv_strategy)

In [ ]:
%%time
grid_cv.fit(filtered, y)

In [ ]:
print grid_cv.best_score_
print grid_cv.best_params_

Поработаем с тестовыми данными

In [6]:
test = pd.read_csv("orange_small_churn_test_data.csv", index_col='ID')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 17.6+ MB


In [14]:
test_numbers = test.ix[:, 0:190]
test_numbers.fillna(0, inplace=True)
test_numbers.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190
ID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,1225.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,896.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,791.0,7.0,0.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,2296.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [15]:
train_numbers = train.ix[:, 0:190]
train_numbers.fillna(0, inplace=True)
train_numbers.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190
ID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,3052.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1813.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,276.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1953.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1533.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,686.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
clf = GradientBoostingClassifier(max_features=0.6, max_depth=3, n_estimators=300, random_state=10)
clf.fit(train_numbers, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=0.6, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=10,
              subsample=1.0, verbose=0, warm_start=False)

In [17]:
result = clf.predict_proba(test_numbers)
result[:5]

array([[ 0.92517532,  0.07482468],
       [ 0.88563352,  0.11436648],
       [ 0.97215905,  0.02784095],
       [ 0.9438115 ,  0.0561885 ],
       [ 0.98422835,  0.01577165]])

In [18]:
def write_result(result, filename):
    churn_probs = [x[1] for x in result]
    ids = range(len(churn_probs))
    df  = pd.DataFrame(index=ids, data=churn_probs, columns=['result'])
    df.to_csv(filename)

In [19]:
write_result(result, "result4.csv")